In [1]:
from tensorflow import keras

imdb=keras.datasets.imdb
(train_data,train_labels),(test_data,test_labels)=imdb.load_data(num_words=10000)

In [2]:
word_index=imdb.get_word_index()
word_index={k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"]=0
word_index["<START>"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3
reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [3]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=256)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=256)

In [4]:
vocab_size=10000
model=keras.Sequential()
model.add(keras.layers.Embedding(vocab_size,16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [6]:
print(train_data.shape,test_data.shape)

x_val=train_data[:10000]
partial_x_train=train_data[10000:]
y_val=train_labels[:10000]
partial_y_train=train_labels[10000:]

(25000, 256) (25000, 256)


In [7]:
history=model.fit(partial_x_train,partial_y_train,epochs=30,batch_size=512,validation_data=(x_val,y_val),verbose=1)

Epoch 1/30
30/30 [==============================] - 1s 14ms/step - loss: 0.6922 - accuracy: 0.5483 - val_loss: 0.6909 - val_accuracy: 0.6045
Epoch 2/30
30/30 [==============================] - 0s 10ms/step - loss: 0.6883 - accuracy: 0.6759 - val_loss: 0.6855 - val_accuracy: 0.7071
Epoch 3/30
30/30 [==============================] - 0s 11ms/step - loss: 0.6801 - accuracy: 0.7241 - val_loss: 0.6752 - val_accuracy: 0.7361
Epoch 4/30
30/30 [==============================] - 0s 10ms/step - loss: 0.6651 - accuracy: 0.7649 - val_loss: 0.6569 - val_accuracy: 0.7569
Epoch 5/30
30/30 [==============================] - 0s 10ms/step - loss: 0.6404 - accuracy: 0.7745 - val_loss: 0.6307 - val_accuracy: 0.7740
Epoch 6/30
30/30 [==============================] - 0s 10ms/step - loss: 0.6076 - accuracy: 0.7954 - val_loss: 0.5967 - val_accuracy: 0.7906
Epoch 7/30
30/30 [==============================] - 0s 10ms/step - loss: 0.5677 - accuracy: 0.8148 - val_loss: 0.5577 - val_accuracy: 0.8045
Epoch 8/30
30

In [8]:
loss,accuracy=model.evaluate(test_data,test_labels)
print("Loss:",loss)
print("Accuracy:",accuracy)

782/782 [==============================] - 1s 2ms/step - loss: 0.3004 - accuracy: 0.8773
Loss: 0.3003775477409363
Accuracy: 0.8773199915885925
